In [26]:
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('stroke')\
    .config("spark.sql.debug.maxToStringFields", "100")\
    .getOrCreate()

In [27]:
# import data with header and auto-datatype
df = spark.read.csv('brain_stroke.csv', header=True, inferSchema=True)

In [28]:
# data glance
df.show(5)

df.columns

+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level| bmi| smoking_status|stroke|
+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
|  Male|67.0|           0|            1|         Yes|      Private|         Urban|           228.69|36.6|formerly smoked|     1|
|  Male|80.0|           0|            1|         Yes|      Private|         Rural|           105.92|32.5|   never smoked|     1|
|Female|49.0|           0|            0|         Yes|      Private|         Urban|           171.23|34.4|         smokes|     1|
|Female|79.0|           1|            0|         Yes|Self-employed|         Rural|           174.12|24.0|   never smoked|     1|
|  Male|81.0|           0|            0|         Yes|      Private|         Urban|           186.

['gender',
 'age',
 'hypertension',
 'heart_disease',
 'ever_married',
 'work_type',
 'Residence_type',
 'avg_glucose_level',
 'bmi',
 'smoking_status',
 'stroke']

In [29]:
df.printSchema()

root
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



In [31]:
from pyspark.sql.functions import col, count, mean, stddev, abs
from pyspark.sql.types import FloatType, DoubleType, IntegerType, LongType, ShortType

# 总行数
total_rows = df.count()

# 计算每列的非空（non-null）值的比例
complete_rates = {column: (df.filter(col(column).isNotNull()).count() / total_rows) * 100 
                  for column in df.columns}

# 输出
print("Complete Rates:")
for column, rate in complete_rates.items():
    print(f"{column}: {rate:.2f}%")

# 计算每列的均值和标准差
from itertools import chain

# 计算每列的均值和标准差
mean_stddev = df.select(*chain(*[(mean(col(column)).alias(f"{column}_mean"), 
                                   stddev(col(column)).alias(f"{column}_stddev")) 
                                  for column in df.columns if df.schema[column].dataType in 
                                  [FloatType(), DoubleType(), IntegerType(), LongType(), ShortType()]]))


# 将结果从 Row 转换为字典以便使用
mean_stddev_values = {**mean_stddev.first().asDict()}

# 计算 Z 分数
# Z = (X-mean)/std
for column in df.columns:
    if df.schema[column].dataType in [FloatType(), DoubleType(), IntegerType(), LongType(), ShortType()]:
        df = df.withColumn(f"{column}_zscore", 
                           (col(column) - mean_stddev_values[f"{column}_mean"]) / 
                           mean_stddev_values[f"{column}_stddev"])

# 指定的阈值,设置为3.2*std
threshold = 3.2

# 计算每个列的离群值的比例
outlier_percentages = {column: (df.filter(col(f"{column}_zscore").isNotNull() & 
                                       (abs(col(f"{column}_zscore")) > threshold)).count() / total_rows) * 100 
                       for column in df.columns if f"{column}_zscore" in df.columns}

# 输出
print("\nOutlier Percentages:")
for column, percentage in outlier_percentages.items():
    print(f"{column}: {percentage:.2f}%")



Complete Rates:
gender: 100.00%
age: 100.00%
hypertension: 100.00%
heart_disease: 100.00%
ever_married: 100.00%
work_type: 100.00%
Residence_type: 100.00%
avg_glucose_level: 100.00%
bmi: 100.00%
smoking_status: 100.00%
stroke: 100.00%
age_zscore: 100.00%
hypertension_zscore: 100.00%
heart_disease_zscore: 100.00%
avg_glucose_level_zscore: 100.00%
bmi_zscore: 100.00%
stroke_zscore: 100.00%

Outlier Percentages:
age: 0.00%
hypertension: 0.00%
heart_disease: 5.52%
avg_glucose_level: 0.50%
bmi: 0.00%
stroke: 4.98%
age_zscore: 0.00%
hypertension_zscore: 0.00%
heart_disease_zscore: 5.52%
avg_glucose_level_zscore: 0.50%
bmi_zscore: 0.00%
stroke_zscore: 4.98%


In [38]:
count_all = df.groupBy(df.columns)\
    .count()\
    .where("count > 1")

print(count_all)
# no duplicates

DataFrame[gender: string, age: double, hypertension: int, heart_disease: int, ever_married: string, work_type: string, Residence_type: string, avg_glucose_level: double, bmi: double, smoking_status: string, stroke: int, age_zscore: double, hypertension_zscore: double, heart_disease_zscore: double, avg_glucose_level_zscore: double, bmi_zscore: double, stroke_zscore: double, age_zscore_zscore: double, hypertension_zscore_zscore: double, heart_disease_zscore_zscore: double, avg_glucose_level_zscore_zscore: double, bmi_zscore_zscore: double, stroke_zscore_zscore: double, count: bigint]
